In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 38.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840628 sha256=6e65fe8bb04ca06abbc61e254af494c8cfbe15f70a2477de5ac2c0fb5f3c4bec
  Stored in directory: /home/karan/.cache/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


Project Description

In [26]:
# A large technology have been hacked! Luckily their engineers have grabbed valuable data about the hacks, including 
# information like session time,locations, wpm typing speed, etc. The engineer relates to you what she has
# been able to figure out so far, she has been able to grab meta data of each session that the hackers used to 
# connect to their servers. These are the features of the data:

#     'Session_Connection_Time': How long the session lasted in minutes
#     'Bytes Transferred': Number of MB transferred during session
#     'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
#     'Servers_Corrupted': Number of server corrupted during the attack
#     'Pages_Corrupted': Number of pages illegally accessed
#     'Location': Location attack came from (Probably useless because the hackers used VPNs)
#     'WPM_Typing_Speed': Their estimated typing speed based on session logs.
        
# The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but
# they aren't very sure if the third hacker was involved or not. We want to figure out whether or not the third 
# suspect had anything to do with the attacks, or was it just two hackers?

# One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have 
# roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each
# should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this 
# is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.

In [3]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("Practise").getOrCreate()

# Verify Spark session creation
print(spark)

your 131072x1 screen size is bogus. expect trouble
24/10/28 08:19:58 WARN Utils: Your hostname, 590-mxl3363mnx resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/28 08:19:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/28 08:19:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.option('header','true').csv("data_kmean.csv",inferSchema=True)

In [7]:
df.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [8]:
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



Generate Summary Statistics for the columns in Dataframe 

In [10]:
df.summary().show()

24/10/28 08:37:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       NULL|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       NULL| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    25%|           

List of Columns

In [11]:
df.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

Define a List of column names that represent features of interest

In [12]:
feature_cols = [
    'Session_Connection_Time',
    'Bytes Transferred',
    'Kali_Trace_Used',
    'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'
]

1) Import The necessary modeules for Creating VectorAssembly 

2) Create a VectorAssembler to assemble selected columns into a feature vector 

3) Transform the DataFrame using the VectorAssembler to add the 'Features' column 

4) Print the schema of dataset

In [16]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=feature_cols,outputCol='Features')

output_data = assembler.transform(df)

output_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- Features: vector (nullable = true)



In [17]:
output_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            Features|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|[31.0,356.32,1.0,...|
|                    2.0|           228.08|              1|             2.48|     

Pre_Processing 
1)  Import the StandScaler class from the Pyspark MLLib library 

2) Create an instance of StandardScaler class 
- 'inputCol' specifies the input to scale ('Features' in this case)
- 'outputCol' specifies the name of the output column ('Features_scaled' in this case)

3) Fit the scaler to the 'output_dataset'
 - This computes summary statistics and prepares the scaler for transformation 


4) Tranform the 'output_dataset' using the fitted scaler
 - This scales the 'features' column and adds a new column 'features_scaled'

In [19]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol= 'Features',
                     outputCol = 'Features_scaled')


scaler_fit_db = scaler.fit(output_data)


scaler_trans_db  = scaler_fit_db.transform(output_data)
scaler_trans_db.head(5)



[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, Features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), Features_scaled=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963])),
 Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, Location='British Virgin Islands', WPM_Typing_Speed=69.08, Features=DenseVector([20.0, 720.99, 0.0, 3.04, 9.0, 69.08]), Features_scaled=DenseVector([1.4196, 2.518, 0.0, 1.3206, 2.9378, 5.151])),
 Row(Session_Connection_Time=31.0, Bytes Transferred=356.32, Kali_Trace_Used=1, Servers_Corrupted=3.71, Pages_Corrupted=8.0, Location='Tokelau', WPM_Typing_Speed=70.58, Features=DenseVector([31.0, 356.32, 1.0, 3.71, 8.0, 70.58]), Features_scaled=DenseVector([2.2004, 1.2444, 1.9976, 1.6117, 2.6114, 5.2628])),
 Row(Session_Connection_Time=2.0, Bytes Transferred=228.08, K

In [20]:
scaler_trans_db.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            Features|     Features_scaled|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58

Find the Optimum Number of Clusters 

1) Import the K-means Class from the Pyspark Mlib Library 

2) Create a Kmean model with 'Features scaled' as the feature column and 'k' clusters set to 2

3) Create a Kmean model with 'Features scaled' as the feature column and 'k' clusters set to 3

4) Fit the first Kmean (kmeans1) to the 'scaler_trans_db' dataset

5) Fit the second Kmean model (kmeans2) to the 'scaler_trans_db' dataset 

6) Get the transing cost (inertia) for the first Kmeans model.
The training cost is a measure  of the sum of sqaured distances from each point to its assigned cluster center 
 


In [22]:
from pyspark.ml.clustering import KMeans 

kmeans1 = KMeans(featuresCol='Features_scaled',k=2)

kmeans2 = KMeans(featuresCol='Features_scaled', k=3)

model_1 = kmeans1.fit(scaler_trans_db)

model_2 = kmeans2.fit(scaler_trans_db)


model_1.summary.trainingCost



24/10/28 09:46:25 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


601.7707512676691

In [23]:
training_cost_2 = model_2.summary.trainingCost
model_2.summary.trainingCost

434.1492898715821

Use the first K-mean (model_1) to make predictions on the 'scaler_trans_db' dataset.
The resulting DataFrame contains a 'prediction' column indicating the cluster assignment 

In [24]:
pre = model_1.transform(scaler_trans_db).select('prediction')

pre.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [25]:
pre = model_2.transform(scaler_trans_db).select('prediction')

pre.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   83|
|         0|   84|
+----------+-----+



Nothing definitive can be said with the above, but the last key fact that the engineer mentioned was that the attacks should be evenly numbered between the hackers! Let's check with the transform and prediction columns that result form this!


It was 2 hackers. In fact, our clustering algorithm created two equally sized clusters with K=2, no way that is a coincidence!